In [ ]:
import flamp
import numpy as np
import numpy.linalg as lin
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import sparse
from tqdm import tqdm

import matrix_functions as mf

flamp.set_dps(100)  # compute with this many decimal digits precision


In [ ]:
dim = 100
rng = np.random.default_rng(42)
lambda_min = 1
kappa = 1_000
lambda_max = kappa * lambda_min

# np.linspace(lambda_min, lambda_max, dim)
a_diag = flamp.linspace(lambda_min, lambda_max, dim)

class DiagMatrix:
    def __init__(self, diag):
        self.diag = diag
        self.shape = (len(self), len(self))

    def __len__(self):
        return len(self.diag)

    def __matmul__(self, other):
        assert len(other) == len(self)
        return self.diag * other

    @property
    def dtype(self):
        return self.diag.dtype

# A = sparse.diags((a_diag), (0))  # this doesn't work with flamp
A = DiagMatrix(a_diag)

In [ ]:
denom_deg = 2
def f(x):
    return x**(-denom_deg)

In [ ]:
x = flamp.to_mp(rng.standard_normal(dim))
ground_truth = mf.diagonal_fa(f, a_diag, x)
krylov_basis, _ = mf.lanczos(A, x, reorthogonalize=True)

In [ ]:
ks = list(range(1, dim, 10))
lanczos_errors = []
krylov_errors = []
cheb_interpolant_errors = []
cheb_regression_errors = []

spectrum_discritization = mf.cheb_nodes(10*dim, a=lambda_min, b=lambda_max)
f_spectrum_discritization = f(spectrum_discritization)
CV = mf.cheb_vandermonde(spectrum_discritization, dim+1)

for k, lanczos_estimate in tqdm(zip(ks, mf.lanczos_fa_multi_k(f, A, x, ks=ks, reorthogonalize=True))):
    lanczos_errors.append(mf.norm(lanczos_estimate - ground_truth))

    # _, squared_l2_error, _, _ = lin.lstsq(krylov_basis[:, :k], ground_truth, rcond=None)
    # krylov_errors.append(np.sqrt(squared_l2_error.item()))
    _, residual = flamp.qr_solve(krylov_basis[:, :k], ground_truth, res=True)
    krylov_errors.append(mf.norm(residual))

    # Degree of polynomial must be strictly less than dimension of Krylov subspace used in Lanczos (so k - 1)
    cheb_interpolant = mf.cheb_interpolation(k - 1, f, lambda_min, lambda_max)
    cheb_interpolant_error = lin.norm(cheb_interpolant(spectrum_discritization) - f_spectrum_discritization, ord=np.inf)
    cheb_interpolant_errors.append(2 * mf.norm(x) * cheb_interpolant_error)

    cheb_coeffs = flamp.qr_solve(CV[:, :k], f_spectrum_discritization)
    # cheb_coeffs, _, _, _ = lin.lstsq(CV[:, :k], f_spectrum_discritization, rcond=None)
    cheb_regression_error = lin.norm(CV[:, :k] @ cheb_coeffs - f_spectrum_discritization, ord=np.inf)
    cheb_regression_errors.append(2 * mf.norm(x) * cheb_regression_error)

results = pd.DataFrame({
    "Number of matrix-vector products": np.array(ks) - 1,
    "Lanczos-FA": np.array(lanczos_errors, float),
    "Krylov subspace": np.array(krylov_errors, float),
    "Our bound": (kappa ** denom_deg) * np.array(krylov_errors, float),
    "Chebyshev interpolant $* 2||x||$": np.array(cheb_interpolant_errors, float),
    "Chebyshev regression $* 2||x||$": np.array(cheb_regression_errors, float)
})

relative_error = False
if relative_error:
    results.loc[:, results.columns != "Number of matrix-vector products"] /= mf.norm(ground_truth)

In [ ]:
results_long = pd.melt(results, ["Number of matrix-vector products"], value_name="Error", var_name="Approximant")
sns.lineplot(x="Number of matrix-vector products", y=("Relative Error" if relative_error else "Error"), hue="Approximant", style="Approximant", data=results_long).set(
    title=f'Approximation of $A^{{-{denom_deg}}}b$',
    yscale='log'
);